In [ ]:
!pip install beautifulsoup4
!pip install requests
!pip install requests-html
!pip install ultralytics

In [ ]:
# Importing packages
import cv2
import torch
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from requests_html import AsyncHTMLSession
import re
import itertools
from PIL import Image
from io import BytesIO
from ultralytics import YOLO

In [ ]:
# Webscraping the images
session = AsyncHTMLSession()

async def scrape_camera_status_data():
    response = await session.get("https://cameras.alertcalifornia.org/?pos=37.2303_-118.9819_6")
    await response.html.arender(sleep=5)
    cameras = response.html.find('.alert-ctt-root.fadein')
    camera_data = []
    for camera in cameras:
        img_tag = camera.find('.alert-ctt-thumb', first=True)
        name_div = camera.find('.alert-ctt-name', first=True)
        name = name_div.text if name_div else 'No Name Found'
        url = img_tag.attrs['src'] if img_tag else 'No URL Found'
        status = 'offline' if 'alert-ctt-stale' in name_div.attrs.get('class', []) else 'online'
        camera_data.append({'name': name, 'status': status, 'url': url})
    df = pd.DataFrame(camera_data)
    return df

camera_df = await scrape_camera_status_data()

camera_df.head(30)

,name,status,url
0,,online,images/blank_thumb.png
1,Alabama Hills 1,online,https://cameras.alertcalifornia.org/public-cam...
2,Alabama Hills 2,online,https://cameras.alertcalifornia.org/public-cam...
3,Aliso Laguna 1,online,https://cameras.alertcalifornia.org/public-cam...
4,Aliso Laguna 2,online,https://cameras.alertcalifornia.org/public-cam...
5,Allen Peak,online,https://cameras.alertcalifornia.org/public-cam...
6,Anaheim Deer Canyon 1,online,https://cameras.alertcalifornia.org/public-cam...
7,Anaheim Deer Canyon 2,online,https://cameras.alertcalifornia.org/public-cam...
8,Antelope Eagle Lake 1,online,https://cameras.alertcalifornia.org/public-cam...
9,Antelope Eagle Lake 2,online,https://cameras.alertcalifornia.org/public-cam...


In [ ]:
# Viewing the data
online_cameras = camera_df[camera_df['status'] == 'online']
online_cameras = online_cameras[1:].reset_index(drop=True)
online_cameras.head(30)

,name,status,url
0,Alabama Hills 1,online,https://cameras.alertcalifornia.org/public-cam...
1,Alabama Hills 2,online,https://cameras.alertcalifornia.org/public-cam...
2,Aliso Laguna 1,online,https://cameras.alertcalifornia.org/public-cam...
3,Aliso Laguna 2,online,https://cameras.alertcalifornia.org/public-cam...
4,Allen Peak,online,https://cameras.alertcalifornia.org/public-cam...
5,Anaheim Deer Canyon 1,online,https://cameras.alertcalifornia.org/public-cam...
6,Anaheim Deer Canyon 2,online,https://cameras.alertcalifornia.org/public-cam...
7,Antelope Eagle Lake 1,online,https://cameras.alertcalifornia.org/public-cam...
8,Antelope Eagle Lake 2,online,https://cameras.alertcalifornia.org/public-cam...
9,Antelope Mtn Susanville,online,https://cameras.alertcalifornia.org/public-cam...


In [ ]:
# Updating the URL's
for name in online_cameras['name']:
    if online_cameras.loc[online_cameras['name'] == name, 'url'].values[0] == "images/blank_thumb.png":
        original_name = online_cameras['name'][0].replace(' ', '')
        new_name = re.sub(r'\s*\(.*?\)\s*', '', name)
        new_name = new_name.replace(' ', '')
        new_url = online_cameras['url'][0].replace(original_name, new_name)
        online_cameras.loc[online_cameras['name'] == name, 'url'] = new_url

online_cameras.head(30)

,name,status,url
0,Alabama Hills 1,online,https://cameras.alertcalifornia.org/public-cam...
1,Alabama Hills 2,online,https://cameras.alertcalifornia.org/public-cam...
2,Aliso Laguna 1,online,https://cameras.alertcalifornia.org/public-cam...
3,Aliso Laguna 2,online,https://cameras.alertcalifornia.org/public-cam...
4,Allen Peak,online,https://cameras.alertcalifornia.org/public-cam...
5,Anaheim Deer Canyon 1,online,https://cameras.alertcalifornia.org/public-cam...
6,Anaheim Deer Canyon 2,online,https://cameras.alertcalifornia.org/public-cam...
7,Antelope Eagle Lake 1,online,https://cameras.alertcalifornia.org/public-cam...
8,Antelope Eagle Lake 2,online,https://cameras.alertcalifornia.org/public-cam...
9,Antelope Mtn Susanville,online,https://cameras.alertcalifornia.org/public-cam...


In [ ]:
# Function to check if the URL returns a 404
def check_url(url):
    try:
        response = requests.get(url)
        if response.status_code == 404:
            return 'image not found'
        else:
            return 'image found'
    except requests.exceptions.RequestException as e:
        return 'error'

# Checking image status
online_cameras['image_status'] = online_cameras['url'].apply(check_url)
online_cameras.head(30)

,name,status,url,image_status
0,Alabama Hills 1,online,https://cameras.alertcalifornia.org/public-cam...,image found
1,Alabama Hills 2,online,https://cameras.alertcalifornia.org/public-cam...,image found
2,Aliso Laguna 1,online,https://cameras.alertcalifornia.org/public-cam...,image found
3,Aliso Laguna 2,online,https://cameras.alertcalifornia.org/public-cam...,image found
4,Allen Peak,online,https://cameras.alertcalifornia.org/public-cam...,image found
5,Anaheim Deer Canyon 1,online,https://cameras.alertcalifornia.org/public-cam...,image found
6,Anaheim Deer Canyon 2,online,https://cameras.alertcalifornia.org/public-cam...,image found
7,Antelope Eagle Lake 1,online,https://cameras.alertcalifornia.org/public-cam...,image found
8,Antelope Eagle Lake 2,online,https://cameras.alertcalifornia.org/public-cam...,image found
9,Antelope Mtn Susanville,online,https://cameras.alertcalifornia.org/public-cam...,image found


In [ ]:
# Function to clean camera names (removing parentheses and special characters)
def clean_name(name):
    name = re.sub(r'\(.*?\)', '', name)
    name = re.sub(r'[^\w\s]', '', name).strip()
    return name

# Function to generate permutations of camera names
def generate_permutations(name):
    words = name.split()
    permutations = []
    for i in range(1, len(words) + 1):
        for combo in itertools.permutations(words, i):
            permutations.append(''.join(combo))
    return permutations

# Function to check if the URL returns a valid image
def check_url(url):
    try:
        response = requests.head(url)
        return response.status_code == 200
    except requests.RequestException:
        return False

In [ ]:
# Updating the URL's
def update_image_status_and_url(df):
    for idx, row in df.iterrows():
        if row['image_status'] == 'image not found':
            original_name = online_cameras['name'][0].replace(' ', '')
            name = clean_name(row['name'])
            permutations = generate_permutations(name)
            for perm in permutations:
                test_url = online_cameras['url'][0].replace(original_name, perm)
                if check_url(test_url):
                    df.at[idx, 'url'] = test_url
                    df.at[idx, 'image_status'] = 'image found'
                    break
    return df

online_cameras_df = update_image_status_and_url(online_cameras)
online_cameras_df.head(30)

,name,status,url,image_status
0,Alabama Hills 1,online,https://cameras.alertcalifornia.org/public-cam...,image found
1,Alabama Hills 2,online,https://cameras.alertcalifornia.org/public-cam...,image found
2,Aliso Laguna 1,online,https://cameras.alertcalifornia.org/public-cam...,image found
3,Aliso Laguna 2,online,https://cameras.alertcalifornia.org/public-cam...,image found
4,Allen Peak,online,https://cameras.alertcalifornia.org/public-cam...,image found
5,Anaheim Deer Canyon 1,online,https://cameras.alertcalifornia.org/public-cam...,image found
6,Anaheim Deer Canyon 2,online,https://cameras.alertcalifornia.org/public-cam...,image found
7,Antelope Eagle Lake 1,online,https://cameras.alertcalifornia.org/public-cam...,image found
8,Antelope Eagle Lake 2,online,https://cameras.alertcalifornia.org/public-cam...,image found
9,Antelope Mtn Susanville,online,https://cameras.alertcalifornia.org/public-cam...,image found


In [ ]:
# Viewing the data
camera_online_df = online_cameras[online_cameras['image_status'] == 'image found']
camera_online_df.head(10)

,name,status,url,image_status
0,Alabama Hills 1,online,https://cameras.alertcalifornia.org/public-cam...,image found
1,Alabama Hills 2,online,https://cameras.alertcalifornia.org/public-cam...,image found
2,Aliso Laguna 1,online,https://cameras.alertcalifornia.org/public-cam...,image found
3,Aliso Laguna 2,online,https://cameras.alertcalifornia.org/public-cam...,image found
4,Allen Peak,online,https://cameras.alertcalifornia.org/public-cam...,image found
5,Anaheim Deer Canyon 1,online,https://cameras.alertcalifornia.org/public-cam...,image found
6,Anaheim Deer Canyon 2,online,https://cameras.alertcalifornia.org/public-cam...,image found
7,Antelope Eagle Lake 1,online,https://cameras.alertcalifornia.org/public-cam...,image found
8,Antelope Eagle Lake 2,online,https://cameras.alertcalifornia.org/public-cam...,image found
9,Antelope Mtn Susanville,online,https://cameras.alertcalifornia.org/public-cam...,image found


In [ ]:
# Importing the model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

/usr/local/lib/python3.10/dist-packages/torch/hub.py:295: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-10-21 Python-3.10.12 torch-2.4.1+cu121 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 202MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [ ]:
# Function to resize the image
def resize_image(image, target_width=600, target_height=350):
    resized_image = cv2.resize(image, (target_width, target_height))
    return resized_image

# Function to check for wildfire-related objects using YOLO
def detect_wildfire(image):
    results = model(image)
    labels = results.names
    detections = results.pred[0]
    wildfire_indicators = ['fire', 'smoke']
    for det in detections:
        class_id = int(det[-1])
        if labels[class_id].lower() in wildfire_indicators:
            return 'wildfire'
    return 'no wildfire'

In [ ]:
# Checking for wildfires in the images
output_data = []

for index, row in online_cameras_df.iterrows():
    name = row['name']
    url = row['url']
    try:
        response = requests.get(url)
        image = Image.open(BytesIO(response.content))
        image = np.array(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        resized_image = resize_image(image)
        wildfire_status = detect_wildfire(resized_image)
        output_data.append({'name': name, 'wildfire_status': wildfire_status})
    except Exception as e:
        output_data.append({'name': name, 'wildfire_status': f'Error: {e}'})


wildfire_df = pd.DataFrame(output_data)
wildfire_df.head()

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.am

,name,wildfire_status
0,Alabama Hills 1,no wildfire
1,Alabama Hills 2,no wildfire
2,Aliso Laguna 1,no wildfire
3,Aliso Laguna 2,no wildfire
4,Allen Peak,no wildfire


In [ ]:
# Checking for wildfires
if not wildfire_df[wildfire_df['wildfire_status'] == 'wildfire'].empty:
    print(wildfire_df[wildfire_df['wildfire_status'] == 'wildfire'])
else:
    print("There are currently no wildfires detected.")

There are currently no wildfires detected.
